In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
sub_Lo = pd.read_csv("../Data/sub_LO.csv")


# London

In [3]:
London = 51.505024,-0.080416

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about

In [6]:
starbucks_Lo = fun.get_results("starbucks", London, 10, 3000, token_foursquare)
df_Lo = fun.clean_results_first_lo(starbucks_Lo, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_Lo = fun.get_results("Nursery school", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results_lo(kinder_Lo, df_Lo, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [8]:
club_Lo = fun.get_results("night club", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results(club_Lo, df_Lo, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_Lo = fun.get_results("vegan", London, 10, 3000, token_foursquare)
df_Lo = fun.clean_results_lo(vegan_Lo, df_Lo, "Vegan restaurant")

- We look for dog hairdressers restaurants around the office and clean the results keeping only the columns that we are interested about

In [10]:
pet_Lo = fun.get_results("dog grooming", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results_lo(pet_Lo, df_Lo, "Pet hairdresser")

- We look for a basketball arena. However, only local UK games are playing nowadays in London. 

In [11]:
Basketball_Lo = fun.get_results("Basketball arena", London, 1, 5000, token_foursquare)
# df_Lo = fun.clean_results_lo(Basketball_Lo, df_Lo, "Basketball arena")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_Lo = fun.get_results_airport("airport", London, 3, 30000, token_foursquare)
df_Lo = fun.clean_results_lo(Airports_Lo, df_Lo, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_Lo = fun.add_company(df_Lo, "1-2 More London Pl, London", "Southwark", 51.505024, -0.080416)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_Lo["Distance"] = df_Lo.apply(fun.calc_distance_LO, axis = 1).round(2)

In [15]:
df_Lo

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"1-2 More London Pl, London",Southwark,51.505024,-0.080416,NaN,0.00
1,Starbucks,"49 Shad Thames, London, Greater London, SE1 2NJ",Null,51.504041,-0.075553,Coffee,354.93
2,Starbucks,"3 Tower Pl (East Building), London, Greater Lo...",Null,51.508576,-0.079207,Coffee,404.00
3,Starbucks,"The Shard Arcade (London Bridge St), London, G...",Null,51.505072,-0.087198,Coffee,470.92
4,Starbucks,"54 Tooley St, Bridge, Greater London, SE1 9SP",Null,51.505487,-0.085133,Coffee,331.54
5,Starbucks,"87 King William St (at Lombard St), London, Gr...",Null,51.512658,-0.088076,Coffee,1002.10
6,Starbucks,"117-119 Houndsditch (Cutler St), London, Great...",Null,51.515524,-0.078639,Coffee,1174.71
7,Starbucks,"223-227 Borough High St, London, Greater Londo...",Null,51.500845,-0.093249,Coffee,1005.08
8,Starbucks,"The Gallery, Bankside 1 (Sumner St), Southwark...",Null,51.506477,-0.100169,Coffee,1380.98
9,Starbucks,"11 Leadenhall St, London, Greater London, EC3V...",Null,51.513328,-0.082907,Coffee,939.93


In [16]:
df_Lo.to_csv("../data/df_Lo.csv", index=False)

In [17]:
Lo_map = Map(location = London, zoom_start = 12)

In [18]:
fun.mapping_results(df_Lo, Lo_map)

In [19]:
fun.mapping_companies(sub_Lo, Lo_map)

In [20]:
Lo_map

In [21]:
Lo_map.save('../images/London_map.html')